# Import all the libraries 

In [1]:
from pymongo import MongoClient
import re
from pymongo import ASCENDING, DESCENDING
import pandas as pd
from pandas.io.json import json_normalize
import requests



# Import the dataset Companies

In [2]:
dbName = "datamad0320"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/datamad0320


# Geocoding

Get the coordinates (longitude and latitude) though an address.

In [3]:
def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}",params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates": [float(data["longt"]), float(data["latt"])]
    }

# What is the city with more Gaming companies?

This could be interresting to see where to install the new office of our Gaming company.

## 1) Get ALL the offices with category Gaming

In [4]:
a = list(db.companies.aggregate([{"$unwind":"$offices"},{"$project":{"offices":1,"name":1,"category_code":1}}]))

In [5]:
df = pd.DataFrame(json_normalize(a))
print(df.shape)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


## 2) Filter only Category Gaming that has the city information

In [6]:
df_gaming = df

df_gaming.drop(df_gaming[df_gaming.category_code != "games_video"].index, axis=0, inplace=True)


In [7]:
df_gaming.shape

(956, 12)

In [8]:
df_gaming.dropna(subset = ['offices.city'])
df_gaming.shape

(956, 12)

## Result: New York 

In [9]:
df_gaming["offices.city"].value_counts()

New York          75
San Francisco     68
                  54
London            36
Los Angeles       31
                  ..
Tenafly            1
Rome               1
2nd Floor          1
Branford           1
Universal City     1
Name: offices.city, Length: 363, dtype: int64

# Designers requirement

Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

In [11]:
df = pd.DataFrame(json_normalize(a))
print(df.shape)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


(16705, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
2,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,52cdef7c4bab8bd675297d8c,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN


## Filter only Category Design that has the city information

In [17]:
df_design = df

df_design.drop(df_design[(df_design['category_code'] != "web") & (df_design['category_code'] != "design")].index , inplace=True)

In [18]:
print(df_design.shape)
df_design.head()

(3085, 12)


,_id,name,category_code,offices.description,offices.address1,offices.address2,offices.zip_code,offices.city,offices.state_code,offices.country_code,offices.latitude,offices.longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
1,52cdef7c4bab8bd675297d8a,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
5,52cdef7c4bab8bd675297d90,Postini,web,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
6,52cdef7c4bab8bd675297d91,Geni,web,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
7,52cdef7c4bab8bd675297d96,Gizmoz,web,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


In [19]:
df_design.dropna(subset = ['offices.city'])
df_design.shape

(3085, 12)

## Result: New York is the second city with more design companies

In [20]:
df_design["offices.city"].value_counts()

San Francisco           217
New York                193
                        184
London                  113
Seattle                  63
                       ... 
Carrboro                  1
Kodambakkam, Chennai      1
Milton Keynes             1
Lipsheim                  1
Dupont                    1
Name: offices.city, Length: 900, dtype: int64

Now that we already have our city we need to continue to investigate in which coordinate we will launch the new office.